In [3]:
from common.imports import *
from common.util import *
from common.const import *
from data.dataloader import *
from model.transformer import *
from model.run_model import *
from model.positonal_encoding import *
from model.standardization import *
from model.batch_processor import *

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
torch.cuda.empty_cache()

cuda


In [3]:
lr = 0.1
max_norm = 100

In [6]:
# OBJECTS

## MODEL
# model = stonks_transformer_model(
#     d_model=D_MODEL,
#     nhead=NUM_HEADS,
#     num_encoder_layers=NUM_ENCODER_LAYERS,
#     num_decoder_layers=NUM_DECODER_LAYERS,
#     dropout=DROPOUT
# ).to(device)
# checkpoint = torch.load("best_model.pt")
# model.load_state_dict(checkpoint['state_dict'])

## DATASET
dataset=ForecasterDataset(0, None)

In [1]:
# CHOOSE TICKER + DAYS_PRED
ticker = "TSLA"
days_pred = 5

In [20]:
idx = HISTORY_DATA().ticker_stoi(ticker)
src = dataset[idx].unsqueeze(0)[:, :, :]
arr = HISTORY_DATA().array_to_dataframe(array=src[0].numpy())
print(arr)

            Open        High         Low       Close       Volume    year  \
0       1.266667    1.666667    1.169333    1.592667  281494496.0  2010.0   
1       1.719333    2.028000    1.553333    1.588667  257806496.0  2010.0   
2       1.666667    1.728000    1.351333    1.464000  123282000.0  2010.0   
3       1.533333    1.540000    1.247333    1.280000   77097000.0  2010.0   
4       1.333333    1.333333    1.055333    1.074000  103003504.0  2010.0   
...          ...         ...         ...         ...          ...     ...   
3260  247.940002  250.970001  244.589996  249.830002  150337904.0  2023.0   
3261  253.509995  259.679993  251.339996  258.709991  162384304.0  2023.0   
3262  260.170013  261.570007  250.500000  256.790009  170575504.0  2023.0   
3263  248.399994  258.950012  247.289993  255.899994  160171200.0  2023.0   
3264  258.920013  263.600006  257.209991  260.540009  167563696.0  2023.0   

      month   day  
0       6.0  29.0  
1       6.0  30.0  
2       7.0   1

In [21]:
src, std_f = standardize(src, torch.Tensor([[src.shape[1]]]))
print(std_f)
arr = HISTORY_DATA().array_to_dataframe(array=src[0].numpy())
print(arr)

(tensor(63.1191), tensor(95810592.), tensor(96.7126), tensor(81401504.))
          Open      High       Low     Close    Volume    year  month   day
0    -0.639549 -0.635413 -0.640555 -0.636178  2.281087  2010.0    6.0  29.0
1    -0.634868 -0.631677 -0.636585 -0.636219  1.990085  2010.0    6.0  30.0
2    -0.635413 -0.634779 -0.638673 -0.637508  0.337480  2010.0    7.0   1.0
3    -0.636791 -0.636723 -0.639749 -0.639411 -0.229892  2010.0    7.0   2.0
4    -0.638859 -0.638859 -0.641734 -0.641541  0.088363  2010.0    7.0   6.0
...        ...       ...       ...       ...       ...     ...    ...   ...
3260  1.911032  1.942361  1.876393  1.930574  0.669856  2023.0    6.0  12.0
3261  1.968625  2.032422  1.946187  2.022392  0.817844  2023.0    6.0  13.0
3262  2.037489  2.051965  1.937502  2.002540  0.918471  2023.0    6.0  14.0
3263  1.915788  2.024874  1.904311  1.993337  0.790656  2023.0    6.0  15.0
3264  2.024564  2.072955  2.006882  2.041315  0.881471  2023.0    6.0  16.0

[3265 rows x 8

In [22]:
src = unstandardize(src, *std_f)
arr = HISTORY_DATA().array_to_dataframe(array=src[0].numpy())
print(arr)

            Open        High         Low       Close       Volume    year  \
0       1.266666    1.666664    1.169338    1.592667  281494496.0  2010.0   
1       1.719337    2.028004    1.553333    1.588673  257806496.0  2010.0   
2       1.666664    1.728001    1.351334    1.464001  123282000.0  2010.0   
3       1.533333    1.540001    1.247330    1.279999   77097000.0  2010.0   
4       1.333332    1.333332    1.055332    1.074001  103003504.0  2010.0   
...          ...         ...         ...         ...          ...     ...   
3260  247.940002  250.970001  244.589996  249.830002  150337904.0  2023.0   
3261  253.509995  259.679993  251.339996  258.709991  162384304.0  2023.0   
3262  260.169983  261.570007  250.500000  256.790009  170575504.0  2023.0   
3263  248.399994  258.950012  247.289993  255.899994  160171200.0  2023.0   
3264  258.920044  263.600006  257.209991  260.540039  167563696.0  2023.0   

      month   day  
0       6.0  29.0  
1       6.0  30.0  
2       7.0   1